In [1]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("JüngerAnalysis").getOrCreate()
spark.sparkContext.setLogLevel('WARN')

In [4]:
text = spark.read.text("The-Forest-Passage.txt").rdd

In [6]:
text.take(20)

[Row(value='Эрнст Юнгер'),
 Row(value=''),
 Row(value='Уход в лес'),
 Row(value=''),
 Row(value=''),
 Row(value=''),
 Row(value=''),
 Row(value='«здесь и сейчас»'),
 Row(value=''),
 Row(value=''),
 Row(value=''),
 Row(value=''),
 Row(value=''),
 Row(value='1'),
 Row(value=''),
 Row(value=''),
 Row(value=''),
 Row(value=''),
 Row(value=''),
 Row(value='Уход в Лес – отнюдь не идиллия скрывается за этим названием. Напротив, читатель должен быть готов к рискованной прогулке не только по проторённым тропам, но и, быть может, уводящей за пределы исследованного.')]

In [7]:
text = text.filter(lambda row: row.value != '')

In [8]:
text.take(10)

[Row(value='Эрнст Юнгер'),
 Row(value='Уход в лес'),
 Row(value='«здесь и сейчас»'),
 Row(value='1'),
 Row(value='Уход в Лес – отнюдь не идиллия скрывается за этим названием. Напротив, читатель должен быть готов к рискованной прогулке не только по проторённым тропам, но и, быть может, уводящей за пределы исследованного.'),
 Row(value='Речь пойдёт о главном вопросе нашего времени, вопросе того рода, что всегда несёт в себе угрозу. Мы задаёмся многими вопросами, также как наши отцы и деды до нас. Между тем, безусловно, значительно изменился смысл того, что называют самим вопросом. Достаточно ли мы осознаём это?'),
 Row(value='Едва ли прошли времена, когда подобные вопросы воспринимались как великие загадки, или Мировые загадки, с оптимизмом, предполагающим уверенность в их разгадке. Некоторые проблемы вообще воспринимались скорее как практические задачи, вроде женского вопроса или социального вопроса. Эти проблемы также считались поддающимися решению, правда, не столько путём изысканий, 

In [9]:
import re
pattern = re.compile("^[1-9][0-9]*$")

text = text.filter(lambda row: not pattern.match(row.value))

In [10]:
text.take(10)

[Row(value='Эрнст Юнгер'),
 Row(value='Уход в лес'),
 Row(value='«здесь и сейчас»'),
 Row(value='Уход в Лес – отнюдь не идиллия скрывается за этим названием. Напротив, читатель должен быть готов к рискованной прогулке не только по проторённым тропам, но и, быть может, уводящей за пределы исследованного.'),
 Row(value='Речь пойдёт о главном вопросе нашего времени, вопросе того рода, что всегда несёт в себе угрозу. Мы задаёмся многими вопросами, также как наши отцы и деды до нас. Между тем, безусловно, значительно изменился смысл того, что называют самим вопросом. Достаточно ли мы осознаём это?'),
 Row(value='Едва ли прошли времена, когда подобные вопросы воспринимались как великие загадки, или Мировые загадки, с оптимизмом, предполагающим уверенность в их разгадке. Некоторые проблемы вообще воспринимались скорее как практические задачи, вроде женского вопроса или социального вопроса. Эти проблемы также считались поддающимися решению, правда, не столько путём изысканий, сколько в ходе ра

In [11]:
text = text.flatMap(lambda row: row.value.split())
text.take(20)

['Эрнст',
 'Юнгер',
 'Уход',
 'в',
 'лес',
 '«здесь',
 'и',
 'сейчас»',
 'Уход',
 'в',
 'Лес',
 '–',
 'отнюдь',
 'не',
 'идиллия',
 'скрывается',
 'за',
 'этим',
 'названием.',
 'Напротив,']

In [12]:
def normalize_string(text):
    symbols = []
    for s in text:
        if s.isalpha():
            symbols.append(s.lower())
    return ''.join(symbols)

text = text.map(normalize_string) \
           .filter(lambda word: word != '')
text.take(20)

['эрнст',
 'юнгер',
 'уход',
 'в',
 'лес',
 'здесь',
 'и',
 'сейчас',
 'уход',
 'в',
 'лес',
 'отнюдь',
 'не',
 'идиллия',
 'скрывается',
 'за',
 'этим',
 'названием',
 'напротив',
 'читатель']

In [14]:
def calculate_count(words):
    words_set = set()
    result = 0
    for word in words:
        words_set.add(word)
        result += 1
    assert len(words_set) == 1
    return result

text.groupBy(lambda word: word) \
    .mapValues(calculate_count) \
    .sortBy(lambda row: row[1], ascending=False) \
    .take(50)

[('в', 63),
 ('и', 52),
 ('что', 34),
 ('не', 30),
 ('на', 23),
 ('к', 21),
 ('мы', 18),
 ('это', 18),
 ('как', 16),
 ('можно', 14),
 ('но', 13),
 ('он', 13),
 ('с', 12),
 ('если', 12),
 ('от', 11),
 ('том', 10),
 ('всё', 10),
 ('при', 10),
 ('же', 10),
 ('только', 9),
 ('они', 9),
 ('то', 9),
 ('этом', 9),
 ('так', 9),
 ('по', 8),
 ('тем', 8),
 ('вопросы', 8),
 ('его', 8),
 ('бы', 8),
 ('быть', 7),
 ('может', 7),
 ('также', 7),
 ('из', 7),
 ('процентов', 7),
 ('или', 6),
 ('их', 6),
 ('этой', 6),
 ('будет', 6),
 ('за', 5),
 ('того', 5),
 ('скорее', 5),
 ('ответ', 5),
 ('избирательный', 5),
 ('бюллетень', 5),
 ('для', 5),
 ('даже', 5),
 ('выборы', 5),
 ('о', 4),
 ('всегда', 4),
 ('нас', 4)]